# MIMIC-IV Examples

Provided below are several examples of clinically meaningful tasks that ACES could easily extract subject cohorts for on the MIMIC-IV dataset. The configuration files for each of these tasks are provided in the repository (`../../../sample_configs`), and cohorts can be extracted using the `aces-cli` tool:

```bash
aces-cli data.path='/path/to/MIMIC/ESGPT/schema/' data.standard='esgpt' cohort_dir='../../../sample_configs' cohort_name='<task_config_name>'
```

A reminder of the common windows that can be specified:

![Window Legend](../assets/windows.png)

In [ ]:
import json

import yaml
from bigtree import print_tree
from omegaconf import DictConfig

from aces import config

In [ ]:
config_path = "../../../sample_configs"

## In-hospital Mortality

![In-hospital Mortality](../assets/inhospital-mortality.png)

In [ ]:
inhospital_mortality_cfg_path = f"{config_path}/inhospital-mortality.yaml"

In [ ]:
with open(inhospital_mortality_cfg_path, "r") as stream:
    data_loaded = yaml.safe_load(stream)
    print(json.dumps(data_loaded, indent=4))

In [ ]:
cfg = config.TaskExtractorConfig.load(config_path=inhospital_mortality_cfg_path)
tree = cfg.window_tree
print_tree(tree)